In [1]:
import pandas as pd
import numpy as np
from LinkInfer import *

from collections import Counter
from scipy import linalg, mat, dot

%load_ext autoreload
%autoreload 2
%load_ext autotime

import time
start = time.time()


In [2]:
eduDf = pd.read_csv('education.csv')
posDf = pd.read_csv('positions.csv')
senDf = pd.read_csv('jobtitle_seniority.csv')

time: 1.53 s


In [3]:
BAwlist = [ "bachelor", "bachelor's", "BACHELOR", "BACHELOR'S", "Bachelor", "Bachelor's", "BA", "B.A.", "B.A", \
          "BS", "B.S.", "BSc", "B.Sc."]
#MAwlist = [ "master", "master's", "MASTER", "MASTER'S", "Master", "Master's", "MA", "M.A.", "M.A", "MSc", "M.S."\
#         "MBA", "M.B.A."]
MBAwlist = ["MBA", 'M.B.A', "M.B.A."]

time: 1.35 ms


In [4]:
eduDf["startyear"] = eduDf["startdate"].str[:4].astype(float)
eduDf["endyear"] = eduDf["enddate"].str[:4].astype(float)
eduDf['major'] = eduDf['major'].fillna("")

eduDf["hasbach"] = eduDf.apply(hasBachDeg, degwlist = BAwlist, mbawlist = MBAwlist, axis = 1) 
#eduDf["hasmas"] = eduDf.apply(hasMasDeg, degwlist = MAwlist, axis = 1) 

time: 23.9 s


In [5]:
#first check the number of unique users in each table

edu_all_user_id = set(eduDf.user_id.unique())
edu_user_id_with_Bach = set(eduDf[eduDf["hasbach"] == True]["user_id"].unique())
edu_user_no_Bach = edu_all_user_id - edu_user_id_with_Bach


pos_all_user_id = set(posDf.user_id.unique())
sen_all_user_id = set(senDf.user_id.unique())

no_Bach_no_pos_user_id = edu_user_no_Bach - pos_all_user_id
print("number of user_id that do not have B.A. and no positions with dates {}".format(len(no_Bach_no_pos_user_id)))

number of user_id that do not have B.A. and no positions with dates 11386
time: 283 ms


In [6]:
print(len(edu_user_id_with_Bach))

45048
time: 1 ms


In [7]:
#calculat eacher user's first bachelor degree start year, if it exist, otherwise, Nan
first_bach_Start = eduDf[eduDf['hasbach']].groupby('user_id').apply(findBachStart)

#generate newDf,  users with bachelor degree, their first bachelor start year, and their age
newDf = pd.DataFrame(data = first_bach_Start).reset_index()
newDf.columns = ['user_id','first_bach_start_year']

newDf['Age'] = (2019 - newDf['first_bach_start_year']) + 18
newDf['T_or_P'] = 'True' #set all to True first
#for those that do not have start date with any of their bachelor degrees, set them to predicted
newDf.loc[newDf['Age'].isnull(),['T_or_P'] ]= "Predicted"

#generate label_fin to be concatenated with other prediction DFs for final_Df later
label_fin = newDf[newDf['T_or_P'] == 'True'][['user_id', 'Age', 'T_or_P']].copy()
print(label_fin.shape)

(39838, 3)
time: 1min 5s


In [8]:
#noticed that the same job title has the same seniority
#outer join seniority and position dfs together
senposDf = senDf.merge(posDf, left_on=['user_id', 'jobtitle'], right_on=['user_id', 'jobtitle'], how = 'outer' )

time: 714 ms


In [9]:
#check number of unique users, 100,000 is correct
senposDf.user_id.nunique()

100000

time: 133 ms


In [10]:
#genrate infoDf - Df with user's first job start year, min and max seniority
#this takes 3 min 
infoDf = senposDf.groupby('user_id').apply(getfirstjob).reset_index()

time: 2min 6s


In [11]:
#generate comDf, merges infoDf with the newDf
comDf = infoDf.merge(newDf, how='left', left_on = 'user_id', right_on= 'user_id')

#fill the na falues of T_or_P with "Predicted"
comDf['T_or_P'] = comDf['T_or_P'].fillna('Predicted')

time: 87.1 ms


In [12]:
#generate the final label user (l_users) id and predction user id sets (p_users)
#there are 39838 user_id with labels
label_user_id = set(comDf[comDf['T_or_P'] == 'True']['user_id'].values)
pred_user_id = edu_all_user_id - label_user_id
print("labeled users {}".format(len(label_user_id)))
print("prediction users {}".format(len(pred_user_id)))

labeled users 39838
prediction users 60162
time: 35.3 ms


In [13]:
#generate Age predictoins for p_users by taking the average age of l_users who have the same first job start year

labelDf = comDf[comDf['user_id'].isin(label_user_id)].copy()
predDf=comDf[comDf['user_id'].isin(pred_user_id)].copy()

#to get the prediction by first_job_year match, merge the prediction and label dfs
labelDf_fby = labelDf[labelDf['first_job_year'].isnull() == False] 
predDf_fby = predDf[predDf.first_job_year.isnull()==False]

m_job_year =   predDf_fby[['user_id','first_job_year',"T_or_P"]].merge(\
    labelDf_fby[['user_id', 'first_job_year','Age']], 
    how = 'left', left_on = 'first_job_year', right_on =  'first_job_year')                                                                    

##for subest of p_users that have first job year:
#make the predictions by finding labeled users who have the same first job year as user, 
#taking mean of all labeled users's age
pred_first_job = m_job_year.groupby(['user_id_x', 'first_job_year', 'T_or_P']).agg({'Age' : 'mean'}).reset_index()
print('nunber of prediction users that can use first job year for prediction') #this should be 46199
print(pred_first_job.user_id_x.nunique())

#there are 31 first job start years that have no other l_users with the same start year, they are quite senior
#array([1884., 1901., 1910., 1913., 1914., 1915., 1917., 1925., 1928.,
#       1931., 1937., 1938., 1939., 1942., 1945., 1949., 1951., 1952.,
#       1956., 1957., 2019.]
#use rule age = first job start year + 22
ttup = pred_first_job[pred_first_job['Age'].isnull()].shape
print("number of outliers {}".format(ttup))
pred_first_job.loc[pred_first_job['Age'].isnull(), 'Age'] = \
    2019- pred_first_job.loc[pred_first_job['Age'].isnull(), 'first_job_year']+22

nunber of prediction users that can use first job year for prediction
46199
number of outliers (31, 4)
time: 33 s


In [14]:
## genreate job_pred_fin, to be concatenated with the label_fin and sen_pred_fin later for final_Df
job_pred_fin = pred_first_job[['user_id_x', 'Age', "T_or_P"]].rename(columns = {'user_id_x' : 'user_id'}).copy()


time: 10.5 ms


In [15]:
#quick check
job_pred_fin.head(2)

,user_id,Age,T_or_P
0,++6+hv3i5RAVsrWO8q5JEQ5+2cvffV/mNepQVJd0smgtpB...,27.88006,Predicted
1,++9DtAOTiRRvECoMIpKbmg4ZM3TcQvn1bQ/jHgHWG0kf/b...,34.36000,Predicted


time: 10.1 ms


In [16]:
## for subest of p_users without first job year, use the most similar l_user in min and max seniority (job title)
# create a set for those p_users ids
pred_first_job_id = set(job_pred_fin['user_id'])
pred_sen_id = pred_user_id - pred_first_job_id
print('number of seniorty prediction users {}'.format(len(pred_sen_id))) #should be 13963

number of seniorty prediction users 13963
time: 19.7 ms


In [17]:
#generate subset of label users Df and seniority prediction users Dfs
comDf_pred_sen = comDf[comDf['user_id'].isin(pred_sen_id)].reset_index(drop=True)
comDf_pred_sen['pred_user_id_idx'] = comDf_pred_sen.index
                    
comDf_label_sen = comDf[comDf['user_id'].isin(label_user_id)].reset_index(drop=True)
comDf_label_sen['label_user_id_idx'] = comDf_label_sen.index

time: 67.6 ms


In [18]:
#quick chek
comDf_pred_sen.tail(3) #check that all is fine

,user_id,first_job_year,min_sen,max_sen,first_bach_start_year,Age,T_or_P,pred_user_id_idx
13960,zzPGPoJIYpHbwq8OAl+mIQ4ZM3TcQvn1bQ/jHgHWG0kf/b...,NaN,5.315111,7.782379,NaN,NaN,Predicted,13960
13961,zzrbQXjc2yHwbWjtQ9F3mg5+2cvffV/mNepQVJd0smgtpB...,NaN,2.490400,3.865393,NaN,NaN,Predicted,13961
13962,zzuZVPanBvW09lNk1C3h+Q5+2cvffV/mNepQVJd0smgtpB...,NaN,1.920548,5.609423,NaN,NaN,Predicted,13962


time: 9.98 ms


In [19]:
##generate the most similar l_user for each p_user, this takes 4 min
id_list = findSenSimId(comDf_pred_sen, comDf_label_sen)

repeat dim 39838
id_list length 13963
time: 3min 38s


In [20]:
#check the results, there should be only 1 l_users that is most similar to more than 100 other users, and his/her age is 30
id_list_cnt = Counter(id_list)
for key, value in id_list_cnt.items():
    if value > 100:
         
        print('most similar id_index {} and number of pred users {}'.format(key, value))
        print("Age of the most 'similar' label user")
        print(comDf_label_sen.loc[key]['Age'])

most similar id_index 224 and number of pred users 197
Age of the most 'similar' label user
30.0
time: 208 ms


In [21]:
#once you have the id_list
comDf_pred_sen['label_id_idx'] = id_list
comDf_pred_sen_M = comDf_pred_sen[['user_id','min_sen', 'max_sen', 'T_or_P', 'label_id_idx']].merge(\
    comDf_label_sen[['label_user_id_idx', 'user_id', 'min_sen', 'max_sen', 'Age']],\
    how = 'left', left_on = 'label_id_idx', right_on = 'label_user_id_idx')

time: 80.1 ms


In [22]:
#quick check
comDf_pred_sen_M.sort_values(by = ["Age"], ascending= False).head(2)

,user_id_x,min_sen_x,max_sen_x,T_or_P,label_id_idx,label_user_id_idx,user_id_y,min_sen_y,max_sen_y,Age
2601,A7juUm89hlsEvzjDvtCpQg5+2cvffV/mNepQVJd0smgtpB...,2.002740,2.875591,Predicted,4137,4137,4chm7maqf1q4va0lUxWbKg5+2cvffV/mNepQVJd0smgtpB...,2.000000,2.901290,137.0
5306,MWMNY+4zE4E/jc0bnpTI4Q5+2cvffV/mNepQVJd0smgtpB...,0.603082,5.112028,Predicted,35926,35926,tgx3Won3qgLg3NHOG04Iwg5+2cvffV/mNepQVJd0smgtpB...,0.580822,5.112028,137.0


time: 21.8 ms


In [23]:
##generate final prediction by seniorty Df to be concatenated with label_fin and job_fin
sen_pred_fin = comDf_pred_sen_M[['user_id_x', 'Age', 'T_or_P']].rename(columns = {'user_id_x': 'user_id'}).copy()

time: 8.95 ms


In [24]:
#quick check, should be 13963
sen_pred_fin.user_id.nunique()

13963

time: 11.5 ms


In [25]:
##generate final Df, concatentae labels and job and seniority prediction
final_Df = pd.concat([label_fin, job_pred_fin, sen_pred_fin], axis = 0).rename(columns ={'T_or_P' : "true or predicted"})
final_Df['Age'] = final_Df['Age'].astype(int)


time: 42.8 ms


In [26]:
#statistics on the overall table
final_Df.Age.describe()

count    100000.00000
mean         38.00758
std          10.79642
min          16.00000
25%          30.00000
50%          35.00000
75%          44.00000
max         157.00000
Name: Age, dtype: float64

time: 12 ms


In [27]:
#ensure no nulls
final_Df.isnull().sum(axis = 0)

user_id              0
Age                  0
true or predicted    0
dtype: int64

time: 19.4 ms


In [28]:
#show final results
final_Df

,user_id,Age,true or predicted
0,++5SW5MI5/h8X1hMA3QnmQ4ZM3TcQvn1bQ/jHgHWG0kf/b...,88,True
1,++5qk2+uEmkI/3Z4FrBwDw4ZM3TcQvn1bQ/jHgHWG0kf/b...,64,True
2,++6zEVtPCi83vpPTHSY2Vg5+2cvffV/mNepQVJd0smgtpB...,36,True
3,++7kB6m0hI1TgAPmyY1X6A5+2cvffV/mNepQVJd0smgtpB...,44,True
4,++8qqHNDQSyp1Yd45ROBZg5+2cvffV/mNepQVJd0smgtpB...,35,True
...,...,...,...
13958,zyPPLml7KIvEhF3Zbi3ZdA5+2cvffV/mNepQVJd0smgtpB...,24,Predicted
13959,zyhR8L/a8lhGdPpcw6waXQ5+2cvffV/mNepQVJd0smgtpB...,48,Predicted
13960,zzPGPoJIYpHbwq8OAl+mIQ4ZM3TcQvn1bQ/jHgHWG0kf/b...,36,Predicted
13961,zzrbQXjc2yHwbWjtQ9F3mg5+2cvffV/mNepQVJd0smgtpB...,31,Predicted


time: 9.99 ms


In [29]:
#save to csv
#final_Df.to_csv("final_Df.csv")

time: 570 µs


In [30]:
#print time
end = time.time()
print("total time for notebook in seconds")
print(end - start)

total time for notebook in seconds
471.24815106391907
time: 1.26 ms
